In [1]:
%load_ext autoreload
%autoreload 2
from __future__ import division, print_function
from svd_improvement import *
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
seed = 417

In [2]:
ratings, users, movies = load_data()
# full matrix is relatively small, so it can be fitted in RAM
R = construct_full_matrix(ratings, users, movies, mode='dense')
train_mask, test_mask = get_train_test_masks(ratings, users, movies)

### Feeding user vectors to autoencoder

In [17]:
rank = 256
U, V = svd_2_factors(R * train_mask, k=rank, method='user_wise')
R_svd = np.dot(U, V.T)

In [12]:
sess, X, X_pred = sma_with_ae(R, train_mask, R_svd, rank)

Epoch: 001, cost = 92863.13281
Epoch: 002, cost = 67105.26562
Epoch: 003, cost = 47402.19531
Epoch: 004, cost = 32711.28516
Epoch: 005, cost = 22039.03516
Epoch: 006, cost = 14489.41211
Epoch: 007, cost = 9290.45703
Epoch: 008, cost = 5806.59668
Epoch: 009, cost = 3535.37207
Epoch: 010, cost = 2095.54053
Epoch: 011, cost = 1207.91309
Epoch: 012, cost = 676.22156
Epoch: 013, cost = 367.38336
Epoch: 014, cost = 193.56589
Epoch: 015, cost = 99.07327
Epoch: 016, cost = 49.36213
Epoch: 017, cost = 24.06050
Epoch: 018, cost = 11.56610
Epoch: 019, cost = 5.61595
Epoch: 020, cost = 2.93596
Epoch: 021, cost = 1.78608
Epoch: 022, cost = 1.32040
Epoch: 023, cost = 1.16333
Epoch: 024, cost = 1.11477
Epoch: 025, cost = 1.09570
Epoch: 026, cost = 1.04994
Epoch: 027, cost = 1.05793
Epoch: 028, cost = 1.02418
Epoch: 029, cost = 1.03963
Epoch: 030, cost = 1.00668
Epoch: 031, cost = 0.99135
Epoch: 032, cost = 0.98574
Epoch: 033, cost = 0.95717
Epoch: 034, cost = 0.97734
Epoch: 035, cost = 0.95076
Epoch:

In [18]:
R_sma = sess.run(X_pred, feed_dict = {X: R})

In [20]:
rmse(R, R_sma, mask=train_mask), rmse(R, R_svd, mask=train_mask)

(0.22207984, 0.33995951979640027)

In [21]:
rmse(R, R_sma, mask=test_mask), rmse(R, R_svd, mask=test_mask)

(0.23326716, 0.64350645592020383)

### Concatenating user and item

In [4]:
rank = 32
U, V = svd_2_factors(R * train_mask, k=rank)
R_svd = np.dot(U, V.T)

In [ ]:
sess, X, X_pred = sma_with_ae(R, train_mask, R_svd, rank, 'ui_concat')

In [ ]:
nnz_indices = np.where(test_mask)
batch_count = int(math.ceil(len(nnz_indices) / batch_size))
u_batch = R.iloc[nnz_indices[0][i*batch_size : (i+1)*batch_size], :].values
i_batch = R.iloc[:, nnz_indices[1][i*batch_size : (i+1)*batch_size]].values.T
X_batch = np.hstack((u_batch, i_batch))

In [ ]:
R_sma = sess.run(X_pred, feed_dict = {X: R})